# PEFT를 활용한 LLM Fine-tuning시키기 (with Transformers🤗 & bitsandbytes)
이 튜토리얼에서는 8비트로 큰 모델을 로드하기 위해 최신 peft 라이브러리를 사용하여 LLM을 파인튜닝하는 방법을 다룰 것입니다. 파인튜닝 방법은 전체 모델의 가중치를 조정하는 대신 어댑터를 튜닝시키고 모델 내부에 적절하게 로드하기만 하면 되는 "LoRA"라는 최신 방법을 사용합니다. 모델을 파인튜닝한 후에는 허깅페이스🤗 허브에서 어댑터를 공유하여 매우 쉽게 로드할 수도 있습니다. 확인해보시죠!

In [ ]:
# 훈련에 필요한 패키지들을 설치합니다 / 지금 버젼은 구글 코랩 사용자를 대상으로 설치를 가이드 합니다.
!pip install transformers[torch] peft datasets sentencepiece evaluate

In [2]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
import sentencepiece

import os

# RLHF에 쓰이는 Reward Model을 파인튜닝 시키기 위한 데이터와 Base Model 준비하기

두 문장을 비교하여 얼마나 유사한지 또는 얼마나 잘 따랐는지(Align) 예측할 수 있는 모델을 만들기 위해 다음과 같은 데이터를 사용하여 2가지 feature를 input으로 받는 LLM을 파인튜닝 시키겠습니다.

In [3]:
train_dataset = load_dataset("fiveflow/cot_ranking", split="train")
eval_dataset = load_dataset("fiveflow/cot_ranking", split="test")

In [ ]:
# 데이터를 파악하니 question과 j,k 응답이 있습니다. 자세한 설명을 드리지 않았지만 지금 사용하는 데이터셋은 저희가 만든 데이터셋이고, LLM이 스스로 J응답이 K응답보다 더 좋다고 판단하여 구축한 데이터입니다.
# 따라서 J응답은 (LLM 기준) 항상 K응답보다 더 나은 Preference를 갖습니다.
train_dataset

In [ ]:
train_dataset[0]

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained("facebook/opt-iml-max-1.3b", num_labels=1),AutoTokenizer.from_pretrained("facebook/opt-iml-max-1.3b")

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-iml-max-1.3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# 데이터셋의 전처리 과정입니다. question에 대한 j응답과 k응답을 비교하고 싶기 때문에 (question + j응답, question + k응답) 쌍으로 데이터를 구축합니다.
"""
num_proc은 인프라 환경에 따라 1로 두셔도 좋습니다.
original columns를 제거하는 이유는 쓸데없는 데이터가 메모리 차지하는 것을 방지하기 위함입니다.
max_length 또한 인프라 환경에 따라 변경하셔도 좋습니다. 단, question과 응답을 합친것이 한 Input이기 때문에 데이터를 확인하여 짤려도 괜찮다고 생각했을 경우 사용하는 것을 권장합니다.
"""

num_proc = 24
original_columns = train_dataset.column_names

def preprocess_function(examples):
    new_examples = {
            "input_ids_j": [],
            "attention_mask_j": [],
            "input_ids_k": [],
            "attention_mask_k": [],
        }
    for question, response_j, response_k in zip(examples["question"], examples["response_j"], examples["response_k"]):
        tokenized_j = tokenizer("Question: " + question + "\n\nAnswer: " + response_j)
        tokenized_k = tokenizer("Question: " + question + "\n\nAnswer: " + response_k)

        new_examples["input_ids_j"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_j"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_k"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_k"].append(tokenized_k["attention_mask"])

    return new_examples


max_length = 1024
train_dataset = train_dataset.map(
    preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns
    )
train_dataset = train_dataset.filter(lambda x: len(x["input_ids_j"]) <= max_length and len(x["input_ids_k"]) <= max_length)

eval_dataset = eval_dataset.map(preprocess_function, batched=True, num_proc=num_proc, remove_columns=original_columns)
eval_dataset = eval_dataset.filter(lambda x: len(x["input_ids_j"]) <= max_length and len(x["input_ids_k"]) <= max_length)

In [6]:
# 다음은 padding 전략입니다. 모든 데이터중 가장 긴 문장에 대해 padding을 하게 된다면 연산이 쓸데 없이 길어질 수 있습니다. 
# 따라서 한 batch안에서 가장 긴 문장에 대해 padding을 한다면 더 효과적일 것 입니다.

from transformers import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy
import evaluate

@dataclass
class RewardDataCollatorWithPadding:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        features_j = []
        features_k = []
        for feature in features:
            features_j.append(
                {
                    "input_ids": feature["input_ids_j"],
                    "attention_mask": feature["attention_mask_j"],
                }
            )
            features_k.append(
                {
                    "input_ids": feature["input_ids_k"],
                    "attention_mask": feature["attention_mask_k"],
                }
            )
        batch_j = self.tokenizer.pad(
            features_j,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch_k = self.tokenizer.pad(
            features_k,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=self.return_tensors,
        )
        batch = {
            "input_ids_j": batch_j["input_ids"],
            "attention_mask_j": batch_j["attention_mask"],
            "input_ids_k": batch_k["input_ids"],
            "attention_mask_k": batch_k["attention_mask"],
            "return_loss": True,
        }
        return batch

In [ ]:
# 메트릭 정의입니다.
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, _ = eval_pred
    predictions = np.argmax(predictions, axis=0)
    labels = np.zeros(predictions.shape, dtype=int)
    return accuracy.compute(predictions=predictions, references=labels)

In [7]:
# 허깅페이스에서 제공하는 trainer 중 loss 부분을 수정하여 사용합니다. 이유는 j응답과 k응답을 비교해야하는데 AutoModelForSequenceClassification으로 불러온 모델을 그대로 사용하기에는 부적절하기 때문입니다.
# 따라서 Input을 두번 받고 그 logit값을 비교하여 j 응답에 더 가깝게 훈련하도록 RLHF에서 제안한 negative log sigmoid함수를 사용하여 다음과 같이 훈련하도록 선언합니다.

from transformers import Trainer
class RewardTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        rewards_j = model(input_ids=inputs["input_ids_j"], attention_mask=inputs["attention_mask_j"])[0]
        rewards_k = model(input_ids=inputs["input_ids_k"], attention_mask=inputs["attention_mask_k"])[0]
        loss = -nn.functional.logsigmoid(rewards_j - rewards_k).mean()
        if return_outputs:
            return loss, {"rewards_j": rewards_j, "rewards_k": rewards_k}
        return loss

# PEFT를 활용하여 훈련 모델 선언하기

지금까지는 RLHF의 Reward Model을 훈련시키기까지 과정을 살펴보았지만, 구글 코랩을 사용하게 되면 opt iml 1.3b모델로도 벅찰 가능성이 매우 높습니다. Reward model이니 어느정도 성능만 나오면 된다 생각할 수 있지만, PPO에 의해 훈련하는 과정에서 reward model의 결과에 좌지우지 될 수 있으므로 Reward Modeling의 정확도는 매우 중요합니다. 따라서 가능한 사용 가능한 파라미터를 늘려 훈련을 꽉꽉 채우기 위해 LoRA를 사용하여 훈련을 시켜보도록 하겠습니다.

In [8]:
# 기존의 AutoModelForSequenceClassification으로 불러온 모델에 LoRA 어댑터를 추가만 하면 되기 때문에 다음과 같은 config를 지정하고 get_peft_model을 사용하여 새로운 모델 선언만 해주면 끝입니다.

"""
task_type은 다양합니다. [CAUSAL_LM, FEATURE_EXTRACTION, QUESTION_ANS, SEQ_2_SEQ_LM, SEQ_CLS, TOKEN_CLS]가 있습니다. 저희는 j,k응답을 비교하여 score를 얻고 싶기 때문에 SEQ_CLS를 사용하겠습니다.
r, lora_alpha, lora_dropout은 pdf에 있는 논문 설명으로 설명을 대체하겠습니다.
"""

from peft import prepare_model_for_int8_training
model = prepare_model_for_int8_training(rank_model)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["j_proj", "k_proj"]
)
lora_model = get_peft_model(model, peft_config)
lora_model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 790,528 || all params: 1,316,548,608 || trainable%: 0.06004548523285515


In [9]:
rank_model

OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(
              in_features=2048, out_features=2048, bias=True
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=2048, out_features=8, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=8, out_features=2048, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
            (v_proj): Line

In [10]:
lora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): OPTForSequenceClassification(
      (model): OPTModel(
        (decoder): OPTDecoder(
          (embed_tokens): Embedding(50272, 2048, padding_idx=1)
          (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
          (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (layers): ModuleList(
            (0-23): 24 x OPTDecoderLayer(
              (self_attn): OPTAttention(
                (k_proj): Linear(
                  in_features=2048, out_features=2048, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=2048, bias=False)
        

In [11]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Colab Notebooks/project/rm_opt',
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    weight_decay=0.001,
    evaluation_strategy="steps",
    save_total_limit = 3,
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    deepspeed=None,
    local_rank=-1,
    remove_unused_columns=False,
    label_names=[],
    logging_strategy="steps",
    logging_steps=10,
    optim="adamw_hf",
    lr_scheduler_type="linear",
)

In [12]:
trainer = RewardTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=RewardDataCollatorWithPadding(tokenizer=tokenizer, max_length=512),
)

In [13]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2640: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored